In [1]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.tensorboard import SummaryWriter

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.autograd as autograd

from easydict import EasyDict as edict

In [2]:
torch.manual_seed(470)
torch.cuda.manual_seed(470)

os.makedirs("images", exist_ok=True)

opt = edict()
opt.n_epochs = 200
opt.batch_size = 64
opt.lr = 0.00005
opt.n_cpu = 4
opt.latent_dim = 100
opt.img_size = 28
opt.channels = 1
opt.clip_value = 0.01
opt.sample_interval = 400
opt.n_critic = 5
opt.b1 = 0.5
opt.b2 = 0.999
print(opt)

{'n_epochs': 200, 'batch_size': 64, 'lr': 5e-05, 'n_cpu': 4, 'latent_dim': 100, 'img_size': 28, 'channels': 1, 'clip_value': 0.01, 'sample_interval': 400, 'n_critic': 5, 'b1': 0.5, 'b2': 0.999}


In [3]:
os.makedirs("../images", exist_ok=True)

In [4]:
img_shape = (opt.channels, opt.img_size, opt.img_size)
img_shape

(1, 28, 28)

In [5]:
cuda = True if torch.cuda.is_available() else False
torch.cuda.is_available()

True

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity


    

In [8]:
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)

100.0%


Extracting ../../data/mnist\MNIST\raw\train-images-idx3-ubyte.gz to ../../data/mnist\MNIST\raw


102.8%


Extracting ../../data/mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ../../data/mnist\MNIST\raw


100.0%


Extracting ../../data/mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data/mnist\MNIST\raw


112.7%
C:\Users\chika\.conda\envs\SlideStyleTransfer\lib\site-packages\torchvision\datasets\mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting ../../data/mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data/mnist\MNIST\raw
Processing...
Done!


In [9]:
dataset = datasets.MNIST("../data/mnist", train=True,
                                  transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]),
                                  download=True)
dataloader = DataLoader(dataset, batch_size=opt.batch_size, shuffle=True)

In [10]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()
generator = generator.cuda() if cuda else generator
discriminator = discriminator.cuda() if cuda else discriminator



In [11]:
# Optimizers
optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=opt.lr)
optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=opt.lr)
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [12]:
batches_done = 0
for epoch in range(opt.n_epochs):
    total_loss_D = 0
    D_num = 0
    total_loss_G = 0
    G_num = 0
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z).detach()
        # Adversarial loss
        loss_D = -torch.mean(discriminator(real_imgs)) + torch.mean(discriminator(fake_imgs))
        total_loss_D += loss_D.item()
        D_num += 1
        loss_D.backward()
        optimizer_D.step()
        
        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-opt.clip_value, opt.clip_value)

        # Train the generator every n_critic iterations
        if i % opt.n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Generate a batch of images
            gen_imgs = generator(z)
            # Adversarial loss
            loss_G = -torch.mean(discriminator(gen_imgs))
            total_loss_G += loss_G.item()
            G_num += 1
            loss_G.backward()
            optimizer_G.step()

    

        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
        batches_done += 1

    print(
        "[Epoch %d/%d] [D loss: %f] [G loss: %f]"
        % (epoch, opt.n_epochs, total_loss_D/D_num, total_loss_G/G_num)
    )

[Epoch 0/200] [D loss: -3.360564] [G loss: -15.418693]
[Epoch 1/200] [D loss: -0.263478] [G loss: -13.386297]
[Epoch 2/200] [D loss: -0.058346] [G loss: -9.253513]
[Epoch 3/200] [D loss: -0.045927] [G loss: -0.221776]
[Epoch 4/200] [D loss: -0.350659] [G loss: -2.477413]
[Epoch 5/200] [D loss: -0.348469] [G loss: -2.598456]
[Epoch 6/200] [D loss: -0.332961] [G loss: -1.901363]
[Epoch 7/200] [D loss: -0.334267] [G loss: -2.054198]
[Epoch 8/200] [D loss: -0.271506] [G loss: -1.458386]
[Epoch 9/200] [D loss: -0.299860] [G loss: -1.206517]
[Epoch 10/200] [D loss: -0.302368] [G loss: -0.893043]
[Epoch 11/200] [D loss: -0.302910] [G loss: -1.006944]
[Epoch 12/200] [D loss: -0.293180] [G loss: -0.769263]
[Epoch 13/200] [D loss: -0.300492] [G loss: -0.748754]
[Epoch 14/200] [D loss: -0.310049] [G loss: -0.531600]
[Epoch 15/200] [D loss: -0.318768] [G loss: -0.430239]
[Epoch 16/200] [D loss: -0.339821] [G loss: -0.373129]
[Epoch 17/200] [D loss: -0.315506] [G loss: -0.327119]
[Epoch 18/200] [D 

KeyboardInterrupt: 

In [19]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [20]:
os.makedirs(f"./log_{0}", exist_ok=True)
log_dir = f"./log_{0}"
writer = SummaryWriter(log_dir)

In [27]:

# Initialize generator and discriminator
generator_1 = Generator()
discriminator_1 = Discriminator()
generator_1 = generator_1.cuda() if cuda else generator_1
discriminator_1 = discriminator_1.cuda() if cuda else discriminator_1

# Loss weight for gradient penalty
lambda_gp = 10
# Optimizers
optimizer_G = torch.optim.Adam(generator_1.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator_1.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))



batches_done = 0
for epoch in range(opt.n_epochs):
    total_loss_D = 0
    D_num = 0
    total_loss_G = 0
    G_num = 0
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        fake_imgs = generator_1(z)

        # Real images
        real_validity = discriminator_1(real_imgs)
        # Fake images
        fake_validity = discriminator_1(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator_1, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty
        
        total_loss_D += d_loss.item()
        D_num +=1

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train the generator every n_critic steps
        if i % opt.n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator_1(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator_1(fake_imgs)
            g_loss = -torch.mean(fake_validity)
            total_loss_G += g_loss.item()
            G_num += 1
            
            g_loss.backward()
            optimizer_G.step()

            if batches_done % opt.sample_interval == 0:
                save_image(fake_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

            batches_done += opt.n_critic

    print("[Epoch %d/%d] [D loss: %f] [G loss: %f]"
    % (epoch, opt.n_epochs, total_loss_D/D_num, total_loss_G/G_num))

[Epoch 0/200] [D loss: -16.846031] [G loss: -11.772812]
[Epoch 1/200] [D loss: -1.663395] [G loss: -15.865155]
[Epoch 2/200] [D loss: -4.386078] [G loss: -4.517751]
[Epoch 3/200] [D loss: -7.235062] [G loss: 0.307346]
[Epoch 4/200] [D loss: -7.404317] [G loss: 0.506245]
[Epoch 5/200] [D loss: -6.656803] [G loss: -0.991276]
[Epoch 6/200] [D loss: -6.549072] [G loss: -0.197864]
[Epoch 7/200] [D loss: -6.342743] [G loss: 1.310993]
[Epoch 8/200] [D loss: -6.515630] [G loss: 1.157407]
[Epoch 9/200] [D loss: -6.098992] [G loss: 1.807060]
[Epoch 10/200] [D loss: -5.826724] [G loss: 1.359904]
[Epoch 11/200] [D loss: -5.616280] [G loss: 1.016793]
[Epoch 12/200] [D loss: -5.606059] [G loss: 0.641945]
[Epoch 13/200] [D loss: -5.449274] [G loss: 0.497028]
[Epoch 14/200] [D loss: -5.391007] [G loss: 0.878521]
[Epoch 15/200] [D loss: -5.330348] [G loss: 1.089084]
[Epoch 16/200] [D loss: -5.115661] [G loss: 1.009703]
[Epoch 17/200] [D loss: -5.140455] [G loss: 1.363624]
[Epoch 18/200] [D loss: -4.928